### Import bibliotek

In [18]:
import requests
from bs4 import BeautifulSoup
import json
import os

### Pobranie ze strony Ceneo.pl opinii o konkretnym produkcie

In [25]:
with open("./headers.json", "r") as jf:
    headers = json.load(jf)

In [26]:
product_id = "175451294"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while(next_page):
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
        for review in reviews:
            try:
                single_review = {
                    "review_id": review["data-entry-id"],
                    "author": review.select_one("span.user-post__author-name").text.strip(),
                    "recomendation": review.select_one("span.user-post__author-recomendation > em").text.strip(),
                    "stars": review.select_one("span.user-post__score-count").text.strip(),
                    "content": review.select_one("div.user-post__text").text.strip(),
                    "pros": [p.text.strip() for p in review.select("div.review-feature__item--positive")],
                    "cons": [c.text.strip() for c in review.select("div.review-feature__item--negative")],
                    "likes": review.select_one("button.vote-yes > span").text.strip(),
                    "dislikes": review.select_one("button.vote-no > span").text.strip(),
                    "publish_date": review.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
                    "purchase_date": review.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip()

                }
                all_reviews.append(single_review)
            except (AttributeError, TypeError): 
                pass
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None
print(all_reviews)

https://www.ceneo.pl/175451294#tab=reviews
2
[{'review_id': '19425088', 'author': 'M...l', 'recomendation': 'Polecam', 'stars': '5/5', 'content': 'MacBook Pro 2024 z M4 to świetne rozwiązanie dla wymagających użytkowników. Wydajny, cichy i energooszczędny – bez problemu radzi sobie z wieloma otwartymi aplikacjami i intensywną pracą. 14-calowy ekran Liquid Retina XDR zachwyca jakością obrazu, a kompaktowy rozmiar i długi czas pracy na baterii czynią go idealnym sprzętem mobilnym. Jeśli ktoś szuka mocy zamkniętej w eleganckiej i lekkiej obudowie, to zdecydowanie warto go rozważyć.', 'pros': ['jakość', 'wydajność', 'wygląd'], 'cons': [], 'likes': '1', 'dislikes': '1', 'publish_date': '2025-02-04 16:28:03', 'purchase_date': '2025-01-28 10:51:01'}, {'review_id': '19391098', 'author': 'k...t', 'recomendation': 'Polecam', 'stars': '5/5', 'content': 'Mistrz w swojej klasie 🤩', 'pros': ['jakość', 'wydajność', 'wygląd'], 'cons': [], 'likes': '1', 'dislikes': '1', 'publish_date': '2025-01-21 19:4


### Zapisanie wszystkich opinii w bazie danych


In [27]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [28]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_reviews, jf, indent=4, ensure_ascii=False)